# Working with Raster Bands

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/planetlabs/notebooks/blob/scipy-2022/jupyter-notebooks/scipy-2022-workshop/2_rasterbands.ipynb)

# Preparing your Workspace 


## Option 1: (recommended) Run in Google Colab

[Open this Notebook in Colab](https://colab.research.google.com/github/planetlabs/notebooks/blob/scipy-2022/jupyter-notebooks/scipy-2022-workshop/2_rasterbands.ipynb), then run the following:

In [ ]:
!pip install rasterio
!wget https://storage.googleapis.com/pdd-stac/disasters/hurricane-harvey/0831/20170831_172754_101c_3B_AnalyticMS.tif
!wget https://storage.googleapis.com/pdd-stac/disasters/hurricane-harvey/0831/20170831_172754_101c_3b_Visual.tif

## Option 2: Run local Jupyter instance

You can also choose to open this Notebook in your own local Jupyter instance. 

**Prerequisites**
- Install: [rasterio](https://pypi.org/project/rasterio) library
- Download data: [20170831_172754_101c_3B_AnalyticMS.tif](https://storage.googleapis.com/pdd-stac/disasters/hurricane-harvey/0831/20170831_172754_101c_3B_AnalyticMS.tif) & [20170831_172754_101c_3b_Visual.tif](https://storage.googleapis.com/pdd-stac/disasters/hurricane-harvey/0831/20170831_172754_101c_3b_Visual.tif)

## Import Packages & Load Data

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

In addition to Rasterio, we'll be working with NumPy, which is a useful package for working with arrays, like our images. We will also be using Matplotlib's Pyplot module to visualize our data.

In [ ]:
image_file = "/content/20170831_172754_101c_3b_Visual.tif"

In [ ]:
satdat = rasterio.open(image_file)

# satdat is our open dataset object
print(satdat)

## Composing a scene

In [ ]:
# Again, let's examine band count and indexes to ensure we have a 3-band visual image
print(satdat.count)
print(satdat.indexes)

Because we know we're look at a PlanetScope 3-band visual satellite image here, we can define the bands by their order:

In [ ]:
# PlanetScope 3-band band order: BGR

blue, green, red = satdat.read()

In [ ]:
# To construct a visual image, we will need the red, green, and blue bands

visual_image = np.dstack((red, green, blue))

## Visualizing an Image

In [ ]:
plt.figure(0)
plt.imshow(visual_image)
plt.title("Visual Image")

Congratulations!! Here we see an image of Houston during floods caused by Hurricane Harvey in August of 2017.